# Naive Bayes Classifier 

## 1) Traning set 불러오기

In [198]:
# training set, test set 만들기
import pandas as pd

data_set = pd.read_csv('./data/model/training_set.csv').dropna()
data_set = data_set[(data_set.lable==1.0) | (data_set.lable==-1.0)]

In [199]:
data_set.shape

(217193, 3)

## 2) Traning

In [200]:
# traning : traning set 으로 토큰별 빈도수 계산
from collections import defaultdict

def set_ngram_list(data_set) :
    # 토큰별로 문서내 빈도수 카운팅
    wordfreq = defaultdict(lambda : [0, 0])
    for i, row in training_set.iterrows():
        words = row['ngrams in doc'].replace('@@@',',').split(',')

        # 문자열 길이가 1인것은 제외
        if len(words[0].split('/')[0]) == 1 :
            continue

        for word in words :
            if row['lable'] == 1 :
                wordfreq[word][1] += 1
            elif row['lable'] == -1 :
                wordfreq[word][0] += 1

        if i % 10000 == 0 :
            print("{} / {}".format(i, training_set.shape[0]))

    pd.DataFrame.from_dict(wordfreq, orient='index').to_csv('./data/model/ngram_list.csv')
    
set_ngram_list(data_set)

50000 / 195473
250000 / 195473
100000 / 195473
220000 / 195473
200000 / 195473
180000 / 195473
280000 / 195473
40000 / 195473
190000 / 195473
210000 / 195473
70000 / 195473
270000 / 195473
30000 / 195473
60000 / 195473
150000 / 195473
140000 / 195473


In [201]:
import math

def predict(test_set, dic_df, sum_1, sum_0, intensity) :
    dic_df["polarity score"] = dic_df["w|pos"]/dic_df["w|neg"]
    dic_df["intensity"] = [x if x > 1 else 1/x for x in dic_df['polarity score']]
    #dic_df["freq"] = dic_df['0']+dic_df['1'] # 전체 토큰별 빈도수 계산
    #dic_df = dic_df.loc[dic_df.freq > 15] 
    dic_df = dic_df.loc[dic_df.intensity>intensity]
    
    #dic_df["w|pos"] += (dic_df['1']+k)/(sum_1+k*2)
    #dic_df["w|neg"] += (dic_df['0']+k)/(sum_0+k*2)
    #dic_df["log(w|pos)"] = np.log(dic_df["w|pos"])
    #dic_df["log(w|neg)"] = np.log(dic_df["w|neg"])

    dic = dic_df.to_dict('index')
    
    prob_pos_tot = math.log(sum_1/(sum_1+sum_0))
    prob_neg_tot = math.log(sum_0/(sum_1+sum_0))
    
    TP = FN = FP = TN = cnt = 0
    predict=[]

    for i, row in test_set.iterrows():
        ngrams = row['ngrams in doc'].split(',')
        lable = row['lable']
        prob_predict = prob_pos = prob_neg = 0

        tmp_token = ""
        tmp_pos = 0
        tmp_neg = 0
        try :
            for token in ngrams :
                if token == '' :
                    continue 
                
                if token in dic.keys() :
                    prob_pos += dic[token]['log(w|pos)']
                    prob_neg += dic[token]['log(w|neg)']
                    
            prob_pos = math.exp(prob_pos_tot + prob_pos)
            prob_neg = math.exp(prob_neg_tot + prob_neg)
        except Exception as e :
            print("===============")
                        
            print(str(e))
            print("prob_pos_tot : {:10.4f}".format(prob_pos_tot))
            print("prob_pos : {:10.4f}".format(prob_pos))
            print("prob_neg_tot : {:10.4f}".format(prob_neg_tot))
            print("prob_neg : {:10.4f}".format(prob_neg))
            print("sum_1 : {:10.4f}".format(sum_1))
            print("sum_0 : {:10.4f}".format(sum_0))
            print(ngrams)


        if prob_pos+prob_neg == 0 :
            prob_predict = 0
        else :
            prob_predict = prob_pos/(prob_pos+prob_neg)

        if prob_predict > 0.5 and lable == 1.0 :
            predict.append(1.0)
            TP += 1
        elif prob_predict > 0.5 and lable == -1.0 :
            predict.append(1.0)
            FP += 1
        elif prob_predict < 0.5 and lable == 1.0 :
            predict.append(-1.0)
            FN += 1
        elif prob_predict < 0.5 and lable == -1.0 :
            predict.append(-1.0)
            TN += 1
            
    print('TP={} / TN={} / FP={} / FN={}'.format(TP, TN, FP, FN))
    #ACC = (TP + TN)  / (전체 데이타 수 = P + N)
    ACC = (TP + TN) / (TP + TN + FP + FN) * 100
    # ERR = (FN+FP) / (전체 데이타수 = P+N)
    ERR = (FN+FP) / (TP + TN + FP + FN) * 100
    # SN = (TP) / P
    SN = (TP) / (TP + FN) * 100
    # PREC = TP / (TP+FP)
    PREC = TP / (TP+FP) * 100
    print("ACC={:7.3f}% / ERR = {:7.3f}% / SN = {:7.3f}% / PREC = {:7.3f}%".format(ACC, ERR, SN, PREC))

In [202]:
#dictionary 파일 저장
import pandas as pd
import numpy as np
from sklearn.utils import shuffle


def traning(data_set) :
    intensity = 1.3
    no_bagging = 30
    k = 0.5
    
    df = pd.read_csv('./data/model/ngram_list.csv', index_col="Unnamed: 0")
    df["freq"] =0
    df["w|pos"] =0
    df["w|neg"] =0

    for i in range(0,no_bagging) :
        print("========== bagging : " + str(i+1))
        data_set = shuffle(data_set)
        training_set = data_set[:int(len(data_set)*0.9)]
        test_set = data_set[int(len(data_set)*0.9):]
        
        sum_1 = df['1'].sum()#training_set.loc[training_set['lable']>0].shape[0]
        sum_0 = df['0'].sum()#training_set.loc[training_set['lable']<0].shape[0]

        df["w|pos"] += (df['1']+k)/(sum_1+k*2)
        df["w|neg"] += (df['0']+k)/(sum_0+k*2)
        df["log(w|pos)"] = np.log(df["w|pos"])
        df["log(w|neg)"] = np.log(df["w|neg"])
        
        predict(test_set, df, sum_1, sum_0, intensity)
 
    df["freq"] = df['0']+df['1'] # 전체 토큰별 빈도수 계산
    #df = df.loc[df.freq > 15] # 빈도가 15 초과인것만 필터링  
    df["w|pos"] = df["w|pos"]/no_bagging
    df["w|neg"] = df["w|neg"]/no_bagging        
    #df["log(w|pos)"] = np.log(df["w|pos"])
    #df["log(w|neg)"] = np.log(df["w|neg"])
    df["polarity score"] = df["w|pos"]/df["w|neg"]
    df["intensity"] = [x if x > 1 else 1/x for x in df['polarity score']]
    df["lable"] = [1 if x > 1 else -1 for x in df['polarity score']]
    df.loc[df.intensity>intensity].to_csv('./data/model/bok_dictionary.csv')
    
traning(data_set)

========== bagging : 1
TP=6764 / TN=8500 / FP=2661 / FN=3795
ACC= 70.276% / ERR =  29.724% / SN =  64.059% / PREC =  71.767%
========== bagging : 2
TP=6678 / TN=8568 / FP=2683 / FN=3791
ACC= 70.193% / ERR =  29.807% / SN =  63.788% / PREC =  71.339%
========== bagging : 3
TP=6730 / TN=8547 / FP=2667 / FN=3776
ACC= 70.336% / ERR =  29.664% / SN =  64.059% / PREC =  71.619%
========== bagging : 4
TP=6812 / TN=8597 / FP=2579 / FN=3732
ACC= 70.944% / ERR =  29.056% / SN =  64.605% / PREC =  72.538%
========== bagging : 5
TP=6699 / TN=8432 / FP=2720 / FN=3869
ACC= 69.664% / ERR =  30.336% / SN =  63.389% / PREC =  71.122%
========== bagging : 6
TP=6788 / TN=8436 / FP=2666 / FN=3830
ACC= 70.092% / ERR =  29.908% / SN =  63.929% / PREC =  71.800%
========== bagging : 7
TP=6899 / TN=8496 / FP=2558 / FN=3767
ACC= 70.879% / ERR =  29.121% / SN =  64.682% / PREC =  72.951%
========== bagging : 8
TP=6825 / TN=8600 / FP=2650 / FN=3645
ACC= 71.017% / ERR =  28.983% / SN =  65.186% / PREC =  72.032%


In [223]:
ff = pd.read_csv('./data/model/bok_dictionary.csv', index_col="Unnamed: 0")
tmp = ff[ff['lable']==1].sort_values('log(w|pos)', ascending=False )
tmp[tmp.index.str.contains(";")].head(30)

,0,1,freq,w|pos,w|neg,log(w|pos),log(w|neg),polarity score,intensity,lable
금리/NNG;인상/NNG,7914,9905,17819,0.000614,0.000456,-3.994450,-4.292904,1.347773,1.347773,1
물가/NNG;안정/NNG,1344,1623,2967,0.000101,0.000077,-5.802956,-6.065578,1.300335,1.300335,1
콜/NNG;금리/NNG;인상/NNG,662,1211,1873,0.000075,0.000038,-6.095681,-6.773335,1.969253,1.969253,1
물가/NNG;상승/NNG,775,946,1721,0.000059,0.000045,-6.342524,-6.615848,1.314325,1.314325,1
ecb/NNG;금리/NNG;인상/NNG,480,911,1391,0.000056,0.000028,-6.380204,-7.094528,2.042806,2.042806,1
인플레이션/NNG;압력/NNG,693,882,1575,0.000055,0.000040,-6.412537,-6.727604,1.370352,1.370352,1
하락/NNG;제한/NNG,639,789,1428,0.000049,0.000037,-6.523896,-6.808669,1.329461,1.329461,1
인플레이션/NNG;억제/NNG,430,647,1077,0.000040,0.000025,-6.722177,-7.204408,1.619685,1.619685,1
서브프라임/NNG;부실/NNG,355,604,959,0.000037,0.000020,-6.790894,-7.395830,1.831136,1.831136,1
가격/NNG;부담/NNG,447,595,1042,0.000037,0.000026,-6.805894,-7.165679,1.433021,1.433021,1


In [224]:
tmp = ff[ff['lable']==-1].sort_values('log(w|neg)', ascending=False )
tmp[tmp.index.str.contains(";")].head(30)

,0,1,freq,w|pos,w|neg,log(w|pos),log(w|neg),polarity score,intensity,lable
금리/NNG;인하/NNG,8425,4743,13168,0.000294,0.000485,-4.730765,-4.230337,0.606271,1.649426,-1
투자/NNG;플러스/NNG,2991,1387,4378,0.000086,0.000172,-5.960037,-5.265825,0.499468,2.002129,-1
경기/NNG;부양/NNG,2428,1655,4083,0.000103,0.000140,-5.783437,-5.474326,0.734100,1.362213,-1
금리/NNG;내리/VV,2147,1319,3466,0.000082,0.000124,-6.010287,-5.597296,0.661668,1.511332,-1
경기/NNG;침체/NNG,1457,1009,2466,0.000063,0.000084,-6.278085,-5.984878,0.745867,1.340721,-1
금리/NNG;낮/VV,1085,738,1823,0.000046,0.000062,-6.590674,-6.279560,0.732630,1.364946,-1
금융시장/NNG;불안/NNG,838,584,1422,0.000036,0.000048,-6.824539,-6.537741,0.750664,1.332155,-1
인하/NNG;효과/NNG,807,323,1130,0.000020,0.000046,-7.416096,-6.575412,0.431415,2.317952,-1
금융시장/NNG;안정/NNG,786,523,1309,0.000032,0.000045,-6.934758,-6.601763,0.716773,1.395141,-1
ecb/NNG;금리/NNG;인하/NNG,747,467,1214,0.000029,0.000043,-7.047896,-6.652621,0.673495,1.484792,-1


## 3) prediction & 검증

In [102]:
def r_squared(y_true, y_hat):
    ssr = 0
    sst = 0
    e = np.subtract(y_true, y_hat)
    y_mean = np.mean(y_true)
    for item in e:
        ssr += item**2
    for item in y_true:
        sst += (item - y_mean)**2
    r2 = 1 - ssr / sst
    return r2

In [103]:
#r_squared(lable_ls, predict)